# Split Video

## Import

In [41]:
import pandas as pd
import os
import sys
sys.path.append('..')
from Module import Result 
from tqdm import tqdm
import numpy as np
import cv2

## Set Config

In [42]:
CONFIG = {
    "data_path" : '../data',
    "target_csv" : "train.csv",
    "split_result_path" : "../data/split_video.result",
    "csv_result_path" : "../data/split_video.csv",
}

## Load CSV

In [43]:
csv_path = os.path.join(CONFIG['data_path'], CONFIG['target_csv'])
data_frame = pd.read_csv(csv_path) # column : sample_id, video_path, label
os.makedirs(CONFIG['split_result_path'], exist_ok=True)

## Split Video

In [44]:
def split_video(data:np.ndarray, result_path:str):
    sample_id = data[0]
    video_path = os.path.join(CONFIG['data_path'], data[1][2:])
    label = data[2]
    result = Result(label=label)
    video = cv2.VideoCapture(video_path)
    frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    results = []
    for frame_idx in range(frame_count):
        _, img = video.read()
        image_name = f"{sample_id}_{label}-{frame_idx}_{result.encoded_crash}_{result.encoded_ego_involve}_{result.encoded_weather}_{result.encoded_timing}.jpg"
        iamge_path = os.path.join(result_path, image_name)
        cv2.imwrite(iamge_path,img)
        results.append({
            'image_name' : image_name,
            'sample_id' : sample_id,
            'label' : label,
            'frame_idx' : frame_idx,
            'crash' : result.encoded_crash,
            'ego_involve' : result.encoded_ego_involve,
            'weather' : result.encoded_weather,
            'timing' : result.encoded_timing,
        })
    return results

In [45]:
results = []
for data in tqdm(data_frame.values):
    [results.append(rs) for rs in split_video(data, CONFIG['split_result_path'])]

100%|██████████| 2698/2698 [57:43<00:00,  1.28s/it] 


## Save Result CSV

In [46]:
results_frame = pd.DataFrame(results)
results_frame.head()
results_frame.to_csv(CONFIG['csv_result_path'], index=False)